# Problem 1: MLP Classifier

Import libraries

In [4]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [5]:
import sklearn.neural_network
import sklearn.model_selection

Load in training and validation data

In [6]:
data_dir = os.path.abspath("data_fashion/")

train_x = pd.read_csv(os.path.join(data_dir, "x_train.csv")).to_numpy()
train_y = pd.read_csv(os.path.join(data_dir, "y_train.csv")).to_numpy()[:,1].astype(np.int32)

valid_x = pd.read_csv(os.path.join(data_dir, "x_valid.csv")).to_numpy()
valid_y = pd.read_csv(os.path.join(data_dir, "y_valid.csv")).to_numpy()[:,1].astype(np.int32)

print(train_x.shape)
print(train_y.shape)

print(valid_x.shape)
print(valid_y.shape)

print(train_y)

(2102, 784)
(2102,)
(600, 784)
(600,)
[2 0 3 ... 5 5 5]


Count number of each class

In [13]:
dress_count_tr = np.count_nonzero(train_y == 3)
dress_count_va = np.count_nonzero(valid_y == 3)
pullover_count_tr = np.count_nonzero(train_y == 2)
pullover_count_va = np.count_nonzero(valid_y == 2)
top_count_tr = np.count_nonzero(train_y == 0)
top_count_va = np.count_nonzero(valid_y == 0)
trouser_count_tr = np.count_nonzero(train_y == 1)
trouser_count_va = np.count_nonzero(valid_y == 1)
sandal_count_tr = np.count_nonzero(train_y == 5)
sandal_count_va = np.count_nonzero(valid_y == 5)
sneaker_count_tr = np.count_nonzero(train_y == 7)
sneaker_count_va = np.count_nonzero(valid_y == 7)

print(dress_count_tr)
print(dress_count_va)
print(pullover_count_tr)
print(pullover_count_va)
print(top_count_tr)
print(top_count_va)
print(trouser_count_tr)
print(trouser_count_va)
print(sandal_count_tr)
print(sandal_count_va)
print(sneaker_count_tr)
print(sneaker_count_va)

400
100
100
100
1
100
1
100
800
100
800
100


Build base model

In [8]:
base_mlp = sklearn.neural_network.MLPClassifier(
    hidden_layer_sizes=(800,),
    activation='relu',
    solver='lbfgs',
    max_iter=10000,
    random_state=1
)

# activation, solver, hidden layer, alpha

Create custom splitter for grid search

In [9]:
# custom splitter code

xall_LF = np.vstack([train_x, valid_x])
yall_L = np.hstack([train_y, valid_y])

valid_indicators_L = np.hstack([
    -1 * np.ones(train_y.size),
    0  * np.ones(valid_y.size),
    ])

my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

Create Grid Searcher

In [10]:
mlp_hyperparam_grid = dict(
    hidden_layer_sizes=[(16,), (32,), (64,), (128,), (256,), (512,), (1024,)],
    alpha=np.logspace(-5, 5, 11)
)

In [11]:
mlp_grid_searcher = sklearn.model_selection.GridSearchCV(
    base_mlp, mlp_hyperparam_grid, scoring='balanced_accuracy', 
    cv=my_splitter, return_train_score=True, refit=False)

Fit grid searcher

In [12]:

mlp_grid_searcher.fit(xall_LF, yall_L)


KeyboardInterrupt: 